# Connors RSI Scoring System with LangGraph
### Connors RSI Score (-100 to +100)

### The Connors RSI score combines three components weighted equally:
##### Price RSI Component (33.33%)
- Based on traditional RSI of closing prices (3-period default)
- Positive when RSI > 50 (bullish momentum)
- Negative when RSI < 50 (bearish momentum)

##### Streak RSI Component (33.33%)
- RSI of consecutive up/down price movements (2-period default)
- Measures persistence of price direction
- Positive when streak RSI > 50 (sustained directional movement)

##### Percent Rank Component (33.33%)
- Percentile ranking of rate of change over 100 periods
- Measures current price change relative to historical context
- Positive when above 50th percentile (above average price movement)

#### Connors RSI Trading Levels
##### Oversold/Overbought Identification:
- Below 20: Oversold (potential buy signal)
- Above 80: Overbought (potential sell signal)
- 20-80: Normal range

#### Score Interpretation:
- Above +60: Strong bullish (price momentum + mean reversion favorable)
- +30 to +60: Moderate bullish
- -30 to +30: Neutral
- -60 to -30: Moderate bearish
- Below -60: Strong bearish

#### Combined Score with Z-Score Enhancement
The system combines Connors RSI with Z-Score analysis for additional confirmation:

#### Z-Score Component:
- Measures how many standard deviations current price is from mean
- Positive when price is above average (bullish bias)
- Negative when price is below average (bearish bias)

#### Combined Score Trading Signals:
- +75 to +100: Strong Buy
- +50 to +75: Buy
- +25 to +50: Weak Buy
- -25 to +25: Neutral (Hold)
- -50 to -25: Weak Sell
- -75 to -50: Sell
- -100 to -75: Strong Sell

In [ ]:
from typing import List, Dict, Any, Optional
from langgraph.graph import StateGraph, END

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI
import pandas as pd
import numpy as np
import yfinance as yf  
import os
from langgraph.prebuilt import ToolNode 
from langchain.agents.agent_types import AgentType
import os
from google.oauth2 import service_account
from dotenv import dotenv_values
import json
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END

# Load environment variables
config = dotenv_values("./keys/.env")
OPENAI_API_KEY = config.get("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Ensure you have your OpenAI API key set as an environment variable
if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

In [ ]:
# Connors RSI Helper Functions
def rsi(series, period=14):
    """Calculate traditional RSI"""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def streak_rsi(series, period=2):
    """Calculate RSI of price streaks"""
    # Calculate streaks
    price_change = series.diff()
    streaks = []
    current_streak = 0
    
    for change in price_change:
        if pd.isna(change):
            streaks.append(0)
            current_streak = 0
        elif change > 0:
            if current_streak >= 0:
                current_streak += 1
            else:
                current_streak = 1
            streaks.append(current_streak)
        elif change < 0:
            if current_streak <= 0:
                current_streak -= 1
            else:
                current_streak = -1
            streaks.append(current_streak)
        else:  # change == 0
            streaks.append(0)
            current_streak = 0
    
    streak_series = pd.Series(streaks, index=series.index)
    return rsi(streak_series, period)

def percent_rank(series, period=100):
    """Calculate percent rank over a rolling window"""
    def rank_pct(x):
        if len(x) < 2:
            return 50.0
        current_value = x.iloc[-1]
        rank = (x < current_value).sum()
        return (rank / (len(x) - 1)) * 100
    
    return series.rolling(window=period).apply(rank_pct, raw=False)

In [ ]:
@tool
def calculate_connors_rsi_score(symbol, period="1y", rsi_period=3, streak_period=2, rank_period=100)->str:
    """
    Calculate a Connors RSI score between -100 and 100 for the given symbol and period.
    
    Connors RSI combines three components:
    1. Price RSI (33.33%): Traditional RSI of closing prices
    2. Streak RSI (33.33%): RSI of consecutive up/down movements  
    3. Percent Rank (33.33%): Percentile ranking of rate of change
    
    Parameters:
    symbol (str): ticker Yahoo finance 
    period (str): Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
    rsi_period (int): Period for price RSI (default 3)
    streak_period (int): Period for streak RSI (default 2)
    rank_period (int): Period for percent rank (default 100)
    
    Returns:
    (str): message with detailed Connors RSI analysis
    """
    
    data = yf.download(symbol, period=period, multi_level_index=False) 
    close = data['Close']
    
    # Component 1: RSI of closing prices (33.33%)
    price_rsi = rsi(close, rsi_period)
    
    # Component 2: RSI of price streaks (33.33%)
    streak_rsi_values = streak_rsi(close, streak_period)
    
    # Component 3: Percent rank of rate of change (33.33%)
    roc = close.pct_change() * 100  # Rate of change as percentage
    percent_rank_values = percent_rank(roc, rank_period)
    
    # Calculate Connors RSI
    crsi = (price_rsi + streak_rsi_values + percent_rank_values) / 3
    
    # Convert Connors RSI to score (-100 to +100)
    # CRSI ranges from 0-100, convert to -100 to +100 scale
    # 50 becomes 0, 0 becomes -100, 100 becomes +100
    connors_score = (crsi - 50) * 2
    
    # Get latest values
    current_crsi = float(crsi.iloc[-1])
    current_score = float(connors_score.iloc[-1])
    current_price_rsi = float(price_rsi.iloc[-1])
    current_streak_rsi = float(streak_rsi_values.iloc[-1])
    current_percent_rank = float(percent_rank_values.iloc[-1])
    
    # Component scores (each worth 33.33% of total)
    price_rsi_score = (current_price_rsi - 50) * 2 * 0.3333
    streak_rsi_score = (current_streak_rsi - 50) * 2 * 0.3333
    percent_rank_score = (current_percent_rank - 50) * 2 * 0.3333
    
    message = f"""
    Symbol: {symbol}, Period: {period}
    Latest Connors RSI: {current_crsi:.2f}
    Latest Connors RSI Score: {current_score:.2f}
    
    Component Analysis:
    1. Price RSI (33.33% weight): {current_price_rsi:.2f} -> Score: {price_rsi_score:.2f}
    2. Streak RSI (33.33% weight): {current_streak_rsi:.2f} -> Score: {streak_rsi_score:.2f}
    3. Percent Rank (33.33% weight): {current_percent_rank:.2f} -> Score: {percent_rank_score:.2f}
    
    Market Condition: {'Oversold' if current_crsi < 20 else 'Overbought' if current_crsi > 80 else 'Normal Range'}
    """
    return message

@tool
def calculate_zscore_indicator(symbol, period="1y", window=20):
    """
    Calculate Z-Score indicator score for mean reversion analysis.
    
    Z-Score measures how many standard deviations the current price is from its mean.
    This helps identify potential mean reversion opportunities.
    
    Parameters:
    symbol (str): ticker Yahoo finance 
    period (str): Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
    window (int): The lookback period for calculating mean and standard deviation
    
    Returns:
    (str): message with Z-Score analysis
    """
    
    data = yf.download(symbol, period=period, multi_level_index=False) 
    close = data['Close']
    
    # Calculate rolling mean and standard deviation
    rolling_mean = close.rolling(window=window).mean()
    rolling_std = close.rolling(window=window).std()
    
    # Calculate Z-Score
    zscore = (close - rolling_mean) / rolling_std
    
    # Convert to score (-100 to +100), capping at 3 standard deviations
    zscore_score = zscore.clip(-3, 3) * (100/3)
    
    # Get latest values
    current_zscore = float(zscore.iloc[-1])
    current_score = float(zscore_score.iloc[-1])
    current_price = float(close.iloc[-1])
    current_mean = float(rolling_mean.iloc[-1])
    current_std = float(rolling_std.iloc[-1])
    
    # Determine mean reversion signal
    if current_zscore > 2:
        reversion_signal = "Strong Mean Reversion Expected (Price very high)"
    elif current_zscore > 1:
        reversion_signal = "Moderate Mean Reversion Expected (Price high)" 
    elif current_zscore < -2:
        reversion_signal = "Strong Bounce Expected (Price very low)"
    elif current_zscore < -1:
        reversion_signal = "Moderate Bounce Expected (Price low)"
    else:
        reversion_signal = "Price near mean (No strong reversion signal)"
    
    message = f"""
    Symbol: {symbol}, Period: {period}, Window: {window}
    Current Price: ${current_price:.2f}
    Rolling Mean: ${current_mean:.2f}
    Current Z-Score: {current_zscore:.2f}
    Z-Score Indicator Score: {current_score:.2f}
    
    Mean Reversion Analysis: {reversion_signal}
    Standard Deviations from Mean: {abs(current_zscore):.2f}
    """
    return message

@tool
def calculate_combined_connors_score(connors_score, zscore_score, connors_weight=0.7, zscore_weight=0.3):
    """
    Calculate a combined score using Connors RSI and Z-Score indicators.
    
    Parameters:
    connors_score (float): Connors RSI score (-100 to +100)
    zscore_score (float): Z-Score indicator score (-100 to +100)
    connors_weight (float): Weight for Connors RSI (default 0.7)
    zscore_weight (float): Weight for Z-Score (default 0.3)
    
    Returns:
    float: Combined score between -100 and 100
    """
    # Weighted combination
    combined = (float(connors_score) * connors_weight) + (float(zscore_score) * zscore_weight)
    return combined

@tool
def interpret_connors_combined_score(score):
    """
    Interpret the combined Connors RSI score and provide trading recommendation.
    
    Args:
    score (float): Combined score between -100 and 100
    
    Returns:
    (str): Trading signal interpretation
    """
    score = float(score)
    if score > 75:
        return "Strong Buy Signal"
    elif score > 50:
        return "Buy Signal"
    elif score > 25:
        return "Weak Buy Signal"
    elif score > -25:
        return "Neutral"
    elif score > -50:
        return "Weak Sell Signal"
    elif score > -75:
        return "Sell Signal"
    else:
        return "Strong Sell Signal"

In [ ]:
# Initialize LLM and tools
llm = ChatOpenAI(model="gpt-4o-mini")
tools = [
    calculate_connors_rsi_score,
    calculate_zscore_indicator, 
    calculate_combined_connors_score,
    interpret_connors_combined_score
]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# State management for LangGraph
from typing import Annotated
from typing_extensions import TypedDict
import operator
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage
from langgraph.graph import StateGraph, START, MessagesState, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

def chatbot(state: State):
    """
    Intelligent trading chatbot that analyzes stocks using Connors RSI and Z-Score indicators.
    
    The bot provides comprehensive analysis by:
    1. Calculating Connors RSI score and interpreting market conditions
    2. Calculating Z-Score for mean reversion analysis  
    3. Combining both indicators with appropriate weights
    4. Providing actionable trading recommendations
    
    Connors RSI Analysis:
    - Combines price momentum, streak analysis, and percentile ranking
    - Identifies oversold (< 20) and overbought (> 80) conditions
    - Provides short-term reversal signals
    
    Z-Score Analysis:
    - Measures price deviation from historical mean
    - Identifies mean reversion opportunities
    - Helps time entries based on statistical extremes
    
    Combined Scoring System:
    +75 to +100: Strong Buy (High momentum + favorable mean reversion)
    +50 to +75: Buy (Good momentum conditions)
    +25 to +50: Weak Buy (Mild bullish signals)
    -25 to +25: Neutral (Hold or wait for clearer signals)
    -50 to -25: Weak Sell (Mild bearish signals)
    -75 to -50: Sell (Poor momentum conditions)
    -100 to -75: Strong Sell (Weak momentum + unfavorable positioning)
    """
    
    # Filter out image messages
    text_messages = [
        msg for msg in state["messages"]
        if not (isinstance(msg.content, list) and msg.content[0].get("type") == "image_url")
    ]
    
    return {"messages": [llm_with_tools.invoke(text_messages)]}

In [ ]:
# Build the graph
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph_builder.add_node("chatbot", chatbot)
tool_node = ToolNode(tools)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_node("tools", tool_node)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
    {"tools": "tools", "__end__": "__end__"},
)

graph = graph_builder.compile()

# Display graph structure (optional)
try:
    from IPython.display import Image, display
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    print("Graph visualization requires additional dependencies")

In [ ]:
def analyze_stock(symbol, period="1y", zscore_window=20):
    """
    Analyze a stock using the Connors RSI system
    """
    config = {"configurable": {"thread_id": "1"}}
    
    message = graph.invoke(
        {
            "messages": [
                (
                    "user",
                    f"""Analyze {symbol} and provide a trading recommendation based on:
                    
                    1. Connors RSI Score calculation (with default parameters: rsi_period=3, streak_period=2, rank_period=100)
                    2. Z-Score indicator analysis (window={zscore_window})
                    3. Combined score calculation (70% Connors RSI, 30% Z-Score)
                    4. Final trading signal interpretation
                    
                    Period: {period}
                    
                    Please provide detailed analysis of each component and a clear recommendation.""",
                )
            ]
        },
        config,
    )
    
    return message["messages"][-1].content

In [ ]:
print("=== UBS Analysis ===")
result = analyze_stock("UBS", period="1y", zscore_window=20)
print(result)

In [ ]:
print("=== TSLA Analysis ===") 
result = analyze_stock("TSLA", period="6mo", zscore_window=30)
print(result)